<a href="https://colab.research.google.com/github/keonju2/sol_ch/blob/main/meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#clone YOLOv5 and 
!git clone https://github.com/hansheng0512/yolov5-tflite
 # clone repo
%cd yolov5-tflite
!pip install -r requirements.txt # install dependencies
!pip install -q roboflow

import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5-tflite'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 58 (delta 4), reused 58 (delta 4), pack-reused 0
Unpacking objects: 100% (58/58), done.
/content/yolov5-tflite
     |████████████████████████████████| 596 kB 7.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 12.3 MB/s 
     |████████████████████████████████| 67 kB 935 kB/s 
     |████████████████████████████████| 138 kB 95.7 MB/s 
     |████████████████████████████████| 145 kB 70.1 MB/s 
     |████████████████████████████████| 178 kB 76.9 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

In [3]:
from zipfile import ZipFile
import os

def unzip(source_file, dest_path):
    with ZipFile(source_file, 'r') as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            try:
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                zf.extract(member, dest_path)
            except:
                raise Exception('what?!')

In [4]:
#!pip install -q wandb
#import wandb
#wandb.login()

In [5]:
file_names = '/content/gdrive/MyDrive/custom_dataset.zip'
unzip(file_names,'/content/yolov5-tflite/custom_dataset')

In [6]:
import shutil
import os

file_source = '/content/yolov5-tflite/custom_dataset/custom_dataset/train/labels'
file_dest = '/content/yolov5-tflite/custom_dataset/custom_dataset/val/labels'
file_dest2 = '/content/yolov5-tflite/data/detect/labels'

get_file = os.listdir(file_source)
for i in get_file:
  if 'Q_0' in i:
    move_file = file_source + '/' + i
    shutil.move(move_file, file_dest2)

In [7]:
file_source = '/content/yolov5-tflite/custom_dataset/custom_dataset/test/labels'
file_dest = '/content/yolov5-tflite/custom_dataset/custom_dataset/val/labels'
file_dest2 = '/content/yolov5-tflite/data/detect/labels'

get_file = os.listdir(file_source)
for i in get_file:
  if 'Q_0' in i or 'Q_1' in i :
    move_file = file_source + '/' + i
    shutil.move(move_file, file_dest2)
  if len(i) == 6:
    move_file = file_source + '/' + i
    shutil.move(move_file, file_dest)

In [8]:
file_name = '/content/gdrive/MyDrive/sc_data_zip/Training/image'
train_list = os.listdir(file_name)
print(train_list)
for i in train_list:
  file_names = file_name+'/'+i
  if '실제도로환경' in i:
      unzip(file_names,'/content/yolov5-tflite/custom_dataset/custom_dataset/train/images')

  elif 'keypoint' in file_names:
      unzip(file_names,'/content/yolov5-tflite/data/detect/images')
  else:
      unzip(file_names,'/content/yolov5-tflite/custom_dataset/custom_dataset/train/images')


['[원천]bbox(실제도로환경).zip', '[원천]bbox(통제환경).zip', '[원천]keypoint(준통제환경).zip']


In [9]:
file_name = '/content/gdrive/MyDrive/sc_data_zip/Validation/image'
train_list = os.listdir(file_name)
print(train_list)

for i in train_list:
  file_names = file_name+'/'+i
  if '[원천]bbox(실제도로환경).zip' in i:
      unzip(file_names,'/content/yolov5-tflite/custom_dataset/custom_dataset/test/images')
  elif 'keypoint' in file_names:
      unzip(file_names,'/content/yolov5-tflite/data/detect/images')
  else:
      unzip(file_names,'/content/yolov5-tflite/custom_dataset/custom_dataset/val/images')


['[원천]bbox(실제도로환경).zip', '[원천]bbox(통제환경).zip', '[원천]keypoint(준통제환경).zip']


In [10]:
import yaml

make_yaml = {
             "train" : "/content/yolov5-tflite/custom_dataset/custom_dataset/train",
            "test": "/content/yolov5-tflite/custom_dataset/custom_dataset/test",
            "val": "/content/yolov5-tflite/custom_dataset/custom_dataset/val",

             "nc" : 4,
             "names": ["left_open_eye","left_closed_eye", "right_open_eye","right_closed_eye"]
}
with open('/content/yolov5-tflite/data/custom_dataset.yaml', 'w') as outfile:
    yaml.dump(make_yaml, outfile, default_flow_style=False)

In [11]:
file_names = '/content/gdrive/MyDrive/hyps.zip'
unzip(file_names,'/content/yolov5-tflite/data')

In [12]:
with open('/content/yolov5-tflite/w+labels.txt','w') as f:
  f.write('left_open_eye\n')
  f.write('left_closed_eye\n')
  f.write('right_open_eye\n')
  f.write('right_closed_eye')

In [18]:
!python train.py --img 320 --cfg yolov5s.yaml --hyp /content/yolov5-tflite/data/hyp.scratch-high.yaml --batch 32 --epochs 1 --data /content/yolov5-tflite/data/custom_dataset.yaml --weights yolov5s.pt --workers 24

train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=/content/yolov5-tflite/data/custom_dataset.yaml, hyp=/content/yolov5-tflite/data/hyp.scratch-high.yaml, epochs=1, batch_size=32, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=24, entity=None, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/hansheng0512/yolov5-tflite ✅
YOLOv5 🚀 52d4d5f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=

In [19]:
save_path = '/content/gdrive/MyDrive/result/fifth'
save_file_path = '/content/yolov5-tflite/runs/train'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/result/fifth'

In [20]:
!python detect.py --source /content/yolov5-tflite/data/detect/images/Q_001_30_M_01_M0_G0_C0 --weights /content/gdrive/MyDrive/result/fifth/exp/weights/best.pt --conf 0.25 --name exp

detect: weights=['/content/gdrive/MyDrive/result/fifth/exp/weights/best.pt'], source=/content/yolov5-tflite/data/detect/images/Q_001_30_M_01_M0_G0_C0, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 52d4d5f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 224 layers, 7062001 parameters, 0 gradients, 16.4 GFLOPs
image 1/25 /content/y

In [ ]:
#!python models/tf.py --weights /content/gdrive/MyDrive/result/fifth/exp/weights/best.pt

In [21]:
save_path = '/content/gdrive/MyDrive/result/fifth/detect'
save_file_path = '/content/yolov5-tflite/runs/detect/exp'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/result/fifth/detect'

In [22]:
!python export.py --weights /content/gdrive/MyDrive/result/fifth/exp/weights/best.pt --include tflite --tf-nms --agnostic-nms

export: data=data/coco128.yaml, weights=/content/gdrive/MyDrive/result/fifth/exp/weights/best.pt, imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=13, tf_nms=True, agnostic_nms=True, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 52d4d5f torch 1.10.0+cu111 CPU

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 224 layers, 7062001 parameters, 0 gradients, 16.4 GFLOPs

PyTorch: starting from /content/gdrive/MyDrive/result/fifth/exp/weights/best.pt (14.3 MB)

TensorFlow saved_model: starting export with tensorflow 2.8.0...

                 from  n    pa

In [25]:
!pip install tflite-support

     |████████████████████████████████| 1.1 MB 7.4 MB/s 
     |████████████████████████████████| 211 kB 78.8 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [29]:
!python metadata_writer_v1.py --model_file /content/gdrive/MyDrive/result/fifth/exp/weights/best-fp16.tflite --label_file /content/yolov5-tflite/w+labels.txt

Metadata populated:
{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
    

In [30]:
save_path = '/content/gdrive/MyDrive/result/fifth/exp/weights'
save_file_path = '/content/yolov5-tflite/best-fp16-metadata-v1.tflite'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/result/fifth/exp/weights/best-fp16-metadata-v1.tflite'